In [1]:
'''
#Merge all files into one
employees = pd.read_csv(PATH+'\\All_Employees.csv', sep='\t')
ch = pd.read_csv(PATH+'\\Course_History.csv', sep='\t')
cd = pd.read_csv(PATH+'\\Course_Definitions.csv', sep='\t')

print(len(employees))
print(len(ch))
print(len(cd))
all_info = pd.merge(ch, cd, how='left', left_on='Course_ID', right_on='Course_ID')
all_info = pd.merge(all_info, employees, how='left', left_on='EID', right_on='EID')

print(len(all_info))
all_info.head()
'''

'''
#Do some basic analysis
#data.Course_Hours.describe()
course_ct = data.loc[:, ['EID', 'Course_ID']].groupby(['EID']).count()
#course_ct.describe()
hrs_per_emp = data.loc[:, ['EID', 'Course_Hours']].groupby(['EID']).sum()
#hrs_per_emp
'''

import os
import numpy as np
import pandas as pd
import random as rnd

PATH = os.getcwd()

data = pd.read_csv(PATH+'\\Horizen_All.txt', sep='\t')
data.head()

,EID,Course_ID,Course_Name,Course_Hours,Course_Date,Last_Name,First_Name,Sex
0,4801,500,What Does this Button Do?,4,2012-03-07,Reyes,Violet,F
1,4801,400,Smart Phones and Dumb People: Know Your Audience,3,2011-02-17,Reyes,Violet,F
2,4801,700,Emotional Intelligence for Employees,3,2020-08-20,Reyes,Violet,F
3,4801,400,Smart Phones and Dumb People: Know Your Audience,3,2019-10-02,Reyes,Violet,F
4,4801,100,Chemtrails and Tin-foil Hats: How to Turn Cons...,3,2022-12-11,Reyes,Violet,F


In [32]:
def insert_nans(dataframe, tgt_col, error_pct):
    df = dataframe.copy()
    
    #Set error percent and total error integer
    error_total = int(round((error_pct * len(df)),0))

    
    #Isolate target column
    tgt_column = df.loc[:, [tgt_col]]
    
    #Set index to save and index to insert nans
    error_idx = rnd.sample(df.index.tolist(), error_total, counts=None)
    good_idx = list(set(tgt_column.index.tolist()) - set(error_idx))
    assert len(good_idx) + len(error_idx) == len(df) #DEBUG
    
    #Separate columns by index
    nan_column = tgt_column.iloc[error_idx, :]
    good_column = tgt_column.iloc[good_idx, :]
    
    #Set all values in NaN column to NaN
    nan_column.iloc[:] = np.nan
    
    #Rejoin columns
    final_column = pd.concat([good_column, nan_column], axis=0)
    assert len(final_column) == len(df) #DEBUG
    
    #Insert finished column into dataframe
    df.loc[:, [tgt_col]] = final_column
    
    return df

In [33]:
#Modify Course Date column
df_new = insert_nans(data, 'Course_Date', 0.1)
len(df_new)

C:\Users\ML2021\AppData\Local\Temp\ipykernel_31516\1609731591.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_column.iloc[:] = np.nan


11540

In [40]:
#Modify Course Hours column
df_new_2 = insert_nans(df_new, 'Course_Hours', 0.05)
len(df_new_2)

C:\Users\ML2021\AppData\Local\Temp\ipykernel_31516\1609731591.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_column.iloc[:] = np.nan


11540

In [41]:
#Output to file
df_new_2.to_csv('Horizen_All_w_Nans.csv', sep='\t', index=False, encoding='utf-8')